---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

# EDA

In [1]:
import pandas as pd
import numpy as np

In [3]:
train_df = pd.read_csv('train.csv', encoding = 'ISO-8859-1')

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
test_df = pd.read_csv('test.csv', encoding = 'ISO-8859-1')

In [4]:
train_df.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [5]:
train_df.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')

In [7]:
train_df.shape

(250306, 34)

In [9]:
test_df.shape

(61001, 27)

In [10]:
test_df.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status'],
      dtype='object')

In [30]:
#agency_name, disposition, fine_amount, admin_fee, state_fee, late_fee, discount_amount, judgment_amount
train_df.grafitti_status.value_counts()

GRAFFITI TICKET    1
Name: grafitti_status, dtype: int64

In [32]:
train_df.isnull().sum()

ticket_id                          0
agency_name                        0
inspector_name                     0
violator_name                     34
violation_street_number            0
violation_street_name              0
violation_zip_code            250306
mailing_address_str_number      3602
mailing_address_str_name           4
city                               0
state                             93
zip_code                           1
non_us_str_code               250303
country                            0
ticket_issued_date                 0
hearing_date                   12491
violation_code                     0
violation_description              0
disposition                        0
fine_amount                        1
admin_fee                          0
state_fee                          0
late_fee                           0
discount_amount                    0
clean_up_cost                      0
judgment_amount                    0
payment_amount                     0
b

In [37]:
# First step remove the NA of compliance

train_df.dropna(subset=['compliance'], inplace=True)

In [51]:
# second step : Include only the columns that appear in the test dataset plus the compliance

keep_col=list(test_df.columns)
keep_col.append('compliance')
keep_col

['ticket_id',
 'agency_name',
 'inspector_name',
 'violator_name',
 'violation_street_number',
 'violation_street_name',
 'violation_zip_code',
 'mailing_address_str_number',
 'mailing_address_str_name',
 'city',
 'state',
 'zip_code',
 'non_us_str_code',
 'country',
 'ticket_issued_date',
 'hearing_date',
 'violation_code',
 'violation_description',
 'disposition',
 'fine_amount',
 'admin_fee',
 'state_fee',
 'late_fee',
 'discount_amount',
 'clean_up_cost',
 'judgment_amount',
 'grafitti_status',
 'compliance']

In [59]:
train_df = train_df[list(keep_col)]


In [65]:
train_df.compliance.value_counts()

0.0    148283
1.0     11597
Name: compliance, dtype: int64

In [64]:
#agency_name, disposition, fine_amount, admin_fee, state_fee, late_fee, discount_amount, judgment_amount
train_df = train_df[['agency_name', 'disposition', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',  'judgment_amount', 'compliance']]

In [70]:
test_df.agency_name.value_counts()

Department of Public Works                        40731
Buildings, Safety Engineering & Env Department    16832
Detroit Police Department                          3438
Name: agency_name, dtype: int64

In [73]:
train_df.disposition.value_counts()

Responsible by Default                138340
Responsible by Admission               13701
Responsible by Determination            7644
Responsible (Fine Waived) by Deter       195
Name: disposition, dtype: int64

In [71]:
# remove Neighborhood City Halls
train_df.agency_name.value_counts()

Buildings, Safety Engineering & Env Department    95863
Department of Public Works                        52445
Health Department                                  7107
Detroit Police Department                          4464
Neighborhood City Halls                               1
Name: agency_name, dtype: int64

In [81]:
train_df = train_df.loc[train_df['agency_name']!='Neighborhood City Halls',]
train_df = train_df.loc[train_df['agency_name']!='Health Department',]

In [82]:
train_df.agency_name.value_counts()

Buildings, Safety Engineering & Env Department    95863
Department of Public Works                        52445
Detroit Police Department                          4464
Name: agency_name, dtype: int64

In [83]:
# make binary the disposition
def disposition_trans(x):
    if x=='Responsible by Default':
        return 1
    else:
        return 0


In [85]:
train_df['disposition'] = train_df['disposition'].apply(lambda x: disposition_trans(x))

In [86]:
test_df['disposition'] = test_df['disposition'].apply(lambda x: disposition_trans(x))

In [88]:
# get dummies
train_agency = pd.get_dummies(train_df.agency_name, prefix='agency')
test_agency = pd.get_dummies(test_df.agency_name, prefix='agency')

In [89]:
train_df_v2 = train_df.join(train_agency)

In [93]:
train_df_v2.drop('agency_name', inplace=True, axis=1)

In [94]:
test_df_v2 = test_df.join(test_agency)
test_df_v2.drop('agency_name', inplace=True, axis=1)

In [95]:
train_df_v2.dtypes

disposition                                                int64
fine_amount                                              float64
admin_fee                                                float64
state_fee                                                float64
late_fee                                                 float64
discount_amount                                          float64
judgment_amount                                          float64
compliance                                               float64
agency_Buildings, Safety Engineering & Env Department      uint8
agency_Department of Public Works                          uint8
agency_Detroit Police Department                           uint8
dtype: object

In [96]:
from sklearn.ensemble import RandomForestClassifier

In [97]:
X_train = train_df_v2.drop('compliance', axis=1)

In [98]:
y_train=train_df_v2.compliance

In [99]:
clf = RandomForestClassifier().fit(X_train, y_train)

In [104]:

from sklearn.metrics import confusion_matrix

In [103]:
predictions = clf.predict(test_df_v2[list(X_train.columns)])

array([ 0.,  0.,  0., ...,  0.,  0.,  1.])

In [105]:
confusion_matrix(y_train, clf.predict(X_train))

array([[141492,    151],
       [  9468,   1661]])

In [106]:
from sklearn.metrics import classification_report

In [107]:
print(classification_report(y_train,clf.predict(X_train)))

             precision    recall  f1-score   support

        0.0       0.94      1.00      0.97    141643
        1.0       0.92      0.15      0.26     11129

avg / total       0.94      0.94      0.92    152772



In [108]:
from sklearn.metrics import roc_curve, auc

In [109]:

from sklearn.model_selection import cross_val_score

In [111]:
cross_val_score(clf, X_train, y_train, cv=5, scoring = 'roc_auc')

array([ 0.7787295 ,  0.79654589,  0.82200387,  0.7628557 ,  0.76274105])

In [112]:
proba = clf.predict_proba(test_df_v2[list(X_train.columns)])

In [115]:
proba = proba[:,1]

In [117]:
proba_series = pd.Series(proba)

In [119]:
proba_series.index=test_df_v2.ticket_id

In [120]:
proba_series

ticket_id
284932    0.051107
285362    0.015040
285361    0.066552
285338    0.051107
285346    0.066552
285345    0.051107
285347    0.055289
285342    0.378336
285530    0.015040
284989    0.026615
285344    0.055289
285343    0.015040
285340    0.015040
285341    0.055289
285349    0.066552
285348    0.051107
284991    0.026615
285532    0.026615
285406    0.026615
285001    0.011740
285006    0.000000
285405    0.015040
285337    0.026615
285496    0.055289
285497    0.051107
285378    0.015040
285589    0.026615
285585    0.051107
285501    0.066552
285581    0.015040
            ...   
376367    0.029802
376366    0.030413
376362    0.189976
376363    0.301282
376365    0.029802
376364    0.030413
376228    0.030413
376265    0.030413
376286    0.293111
376320    0.030413
376314    0.030413
376327    0.293111
376385    0.293111
376435    0.419784
376370    0.293111
376434    0.073812
376459    0.050284
376478    0.000000
376473    0.030413
376484    0.014510
376482    0.011740
37

In [121]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    
    return proba_series

In [122]:
blight_model()

ticket_id
284932    0.051107
285362    0.015040
285361    0.066552
285338    0.051107
285346    0.066552
285345    0.051107
285347    0.055289
285342    0.378336
285530    0.015040
284989    0.026615
285344    0.055289
285343    0.015040
285340    0.015040
285341    0.055289
285349    0.066552
285348    0.051107
284991    0.026615
285532    0.026615
285406    0.026615
285001    0.011740
285006    0.000000
285405    0.015040
285337    0.026615
285496    0.055289
285497    0.051107
285378    0.015040
285589    0.026615
285585    0.051107
285501    0.066552
285581    0.015040
            ...   
376367    0.029802
376366    0.030413
376362    0.189976
376363    0.301282
376365    0.029802
376364    0.030413
376228    0.030413
376265    0.030413
376286    0.293111
376320    0.030413
376314    0.030413
376327    0.293111
376385    0.293111
376435    0.419784
376370    0.293111
376434    0.073812
376459    0.050284
376478    0.000000
376473    0.030413
376484    0.014510
376482    0.011740
37